In [ ]:
from pathlib import Path

import awkward as ak
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import tqdm.notebook as tqdm
import uproot

run_name = "run_050016_10192021_21h49min_Ascii_build"
# run_name = "build"
raw_path = Path("data/raw") / f"{run_name}.root"
img_path = Path("img/coverage")
img_path.mkdir(exist_ok=True, parents=True)


b_cosmic = uproot.open(raw_path)
print("trees:", b_cosmic.keys())
tree = b_cosmic["ecal"]
print(len(tree))
tree.show()

In [ ]:
x, y, z, is_hit, is_masked, is_commissioned = ak.unzip(
    tree.arrays(filter_name=["hit_[xyz]", "hit_is*"], entry_stop=1e6)
)

In [ ]:
tree.num_entries

In [ ]:
n_layers = 15
distance_layers_mm = 15

xy_pos = np.linspace(3.8, 86.3, 16)
xy = np.concatenate([-xy_pos[::-1], xy_pos])
delta_xy = np.mean(xy_pos[1:] - xy_pos[:-1])
xy_bins_pos = np.concatenate([[xy_pos[0] - delta_xy / 2], xy_pos + delta_xy / 2])
xy_bins = np.concatenate([-xy_bins_pos[::-1], xy_bins_pos])

layer_maps = {
    i_z: np.zeros(2 * (len(xy) + 1,))
    for i_z in distance_layers_mm * np.arange(n_layers)
}
layer_maps_is_hit = {
    i_z: np.zeros(2 * (len(xy) + 1,))
    for i_z in distance_layers_mm * np.arange(n_layers)
}
with tqdm.tqdm(unit="M events", total=tree.num_entries / 1e6) as p_bar:
    for batch in uproot.iterate(
        tree, filter_name=["hit_[xyz]", "hit_isHit"], step_size="10 MB"
    ):
        x, y, z, is_hit = ak.unzip(batch)
        for i_z in layer_maps:
            layer_maps[i_z] += np.histogram2d(
                *map(lambda v: ak.flatten(v[z == i_z]).to_numpy(), [x, y]), bins=xy_bins
            )[0]
            layer_maps_is_hit[i_z] += np.histogram2d(
                *map(
                    lambda v: ak.flatten(v[(z == i_z) & (is_hit == 1)]).to_numpy(),
                    [x, y],
                ),
                bins=xy_bins,
            )[0]
        p_bar.update(len(z) / 1e6)

In [ ]:
len(z)

In [ ]:
def _my_format(val):
    """Enforce a format style with 5 digits maximum including the decimal dot."""
    if val < 1e2:
        return f"{val:.2f}"
    if val < 1e3:
        return f"{val:.1f}"

    if val < 1e6:
        v_new, suffix = val / 1e3, "k"
    elif val < 1e9:
        v_new, suffix = val / 1e6, "M"
    elif val < 1e12:
        v_new, suffix = val / 1e9, "B"
    else:
        raise NotImplementedError(f"Value is larger than forseen: {val}.")

    if v_new < 10:
        return f"{v_new:.2f}{suffix}"
    elif v_new < 100:
        return f"{v_new:.1f}{suffix}"
    else:
        return f"{v_new:.0f}{suffix}"


def hit_map(layer_counts, log_norm=False, with_numbers=True):
    norm_type = colors.LogNorm if log_norm else colors.Normalize
    max_counts = max(np.max(v) for v in layer_counts.values())
    max_counts = 20000
    norm = norm_type(vmin=0, vmax=max_counts)
    fig, axs = plt.subplots(1 + (len(layer_counts) - 1) // 5, ncols=5, figsize=(15, 10))
    for i, z in enumerate(layer_counts.values()):
        ax = axs.flatten()[i]
        ax.imshow(z.T, cmap=plt.get_cmap("Greens"), norm=norm)
        if with_numbers:
            boxes = [(0, 8), (8, 16), (17, 25), (25, 33)]
            for x_box in boxes:
                for y_box in boxes:
                    box_vals = z[x_box[0] : x_box[1], y_box[0] : y_box[1]]
                    val = np.mean(box_vals)
                    assert np.std(box_vals) == 0
                    color = "black"
                    if val > 0.7 * max_counts:
                        color = "white"
                    elif log_norm and np.log(val) > 0.7 * np.log(max_counts):
                        color = "white"
                    ax.text(
                        np.mean(x_box),
                        np.mean(y_box),
                        _my_format(val),
                        ha="center",
                        va="center",
                        color=color,
                        fontsize=14,
                    )
        ax.set_title(f"layer {i}")
        ax.set_xticks([])
        ax.set_yticks([])
    fig.tight_layout()
    return fig


for log_norm in [True, False]:
    name = "hit_map"
    if log_norm:
        name += "_log"
    fig = hit_map(layer_maps, log_norm=log_norm)
    fig.savefig(img_path / f"{name}.png", dpi=300)
    name += "_isHit"
    fig = hit_map(layer_maps_is_hit, with_numbers=False, log_norm=log_norm)
    fig.savefig(img_path / f"{name}.png", dpi=300)

In [ ]:
counts = np.concatenate([v.flatten() for v in layer_maps_is_hit.values()])
bins = np.concatenate(
    [
        np.arange(0, 10, 1),
        np.exp(np.linspace(np.log(10), np.log(max(counts)), 100)),
    ]
)
hist_data, _ = np.histogram(counts, density=True, bins=bins)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, sharex="col", sharey="row")
bin_centers = (bins[1:] + bins[:-1]) / 2
bin_widths = bins[1:] - bins[:-1]
for is_log_x in [True, False]:
    for is_log_y in [True, False]:
        ax = axs[int(is_log_y), int(is_log_x)]
        ax.bar(bin_centers, hist_data, width=bin_widths)
        if is_log_x:
            ax.set_xscale("log")
        else:
            ax.set_xlim(0, 2000)
        if is_log_y:
            ax.set_yscale("log")
            ax.set_ylim((0.5 * min(hist_data[hist_data > 0]), 1.1 * max(hist_data)))
        else:
            ax.set_ylim(0, 0.003)
        lam = np.mean(counts)
        ax.plot(
            bin_centers,
            bin_widths / sum(bin_widths) / lam * np.exp(-bin_centers / lam),
            ls=":",
            color="black",
        )

fig.suptitle(
    f"{100 * sum(counts == 0) / len(counts):.3g}% of cells were never hit\n"
    f"({int(sum(counts))} hits in {len(counts)} cells)"
)
axs[1][0].set_xlabel("cell hits [linear, lower part]")
axs[1][1].set_xlabel("cell hits [log]")
axs[0][0].set_ylabel("[linear, lower part]")
axs[1][0].set_ylabel("cell hit density [log]")
fig.tight_layout()
fig.savefig(img_path / "cell_counts.png", dpi=400)

In [ ]:
plt.hist(counts, density=True, bins=np.exp(np.log(np.arange(1, max(counts), 20))))
plt.title(
    f"{100 * sum(counts == 0) / len(counts):.3g}% of cells were never hit\n({int(sum(counts))} hits in {len(counts)} cells)"
)
plt.xscale("log")

In [ ]:
ak.mean(is_hit), ak.mean(is_masked), ak.mean(is_commissioned)

In [ ]:
print(f"{ak.count(is_hit):>10} all")
print(f"{ak.sum(is_hit == 1):>10} is_hit")
print(f"{ak.sum(is_masked == 1):>10} is_masked")
print(f"{ak.sum(is_commissioned == 1):>10} is_commissioned")